# 본문 스크래핑

In [1]:
import pandas as pd
import numpy as np
import time
from tqdm import tqdm
import random
import urllib.request
from selenium.common.exceptions import NoSuchElementException, UnexpectedAlertPresentException, TimeoutException
from selenium import webdriver
from selenium.webdriver.common.by import By
# import chromedriver_auto


# 웹드라이버 설정
options = webdriver.ChromeOptions()
options.add_experimental_option("excludeSwitches", ["enable-automation"])
options.add_experimental_option("useAutomationExtension", False)
options.add_argument('headless') # 창 숨기기


In [2]:
# from user_agent import generate_user_agent, generate_navigator

# print(generate_user_agent(device_type='desktop'))
# print(generate_user_agent(os='win', device_type='desktop'))

# navigator = generate_navigator()

# random_user = generate_user_agent(os='win', device_type='desktop')
# print(random_user)

In [2]:
from selenium.webdriver.chrome.options import Options

# chrome_options = Options()
# chrome_options.add_argument(random_user)
driver = webdriver.Chrome(executable_path ='../chromedriver_win32/chromedriver.exe',options = options)

driver.maximize_window()
driver.get("https://www.naver.com/")
driver.implicitly_wait(10)


In [3]:
url_df = pd.read_csv("../dataset/2022/링크_크롤링/나무문화_링크_2022_7월.csv", encoding='UTF-8')
print(url_df.shape, url_df.columns)
url_df


(18989, 3) Index(['url', 'title', 'date'], dtype='object')


,url,title,date
0,https://blog.naver.com/kuna611/222835896356,"[블챌] 7월 4주차_주간 일기? 일상 기록이라 쓰고, 욕망의...",2022.07.31.
1,https://blog.naver.com/zjavbxjqnd/222835899401,안동 여행지 추천! (in.월영교/낙강물길공원/병산서원),2022.07.31.
2,https://blog.naver.com/roseyh78/222835897663,경주 건천맛집 명문갈비 가야밀면 편백나무숲 맛집,2022.07.31.
3,https://blog.naver.com/y_girok_/222835898249,[주간일기 챌린지 / 7월 4주차] 7월 25일 ~ 7월 31일,2022.07.31.
4,https://blog.naver.com/anuyheel/222835899414,[수영구 남천동 / 광안리 화실] 보통날의 작업실 - 아크릴...,2022.07.31.
...,...,...,...
18984,https://blog.naver.com/so9650/222795579526,[2022년 6월] 동서고금을 막론하고,2022.07.01.
18985,https://blog.naver.com/ottffss758/222794556276,"안동 하루 여행 걷기 좋은 월영교, 안동여행 1편",2022.07.01.
18986,https://blog.naver.com/sil8004/222761395361,영주 여행지/소수서원~한국 유교 문화 발상지,2022.07.01.
18987,https://blog.naver.com/cool2404/222795339338,"단 하나, 트리니티클럽",2022.07.01.


In [4]:
blog_links = url_df['url'].to_list()
print(len(blog_links))


18989


In [7]:
# df = pd.DataFrame(columns = ['url','본문'])
# df.to_csv("../dataset/2022/본문_크롤링/나무문화_본문_2022_7월.csv", index=False, encoding='UTF-8')
#df.to_excel("../dataset/2022/본문_크롤링/나무문화_본문_2022_7월.xlsx", index=False, encoding='UTF-8')

In [5]:
df = pd.read_csv("../dataset/2022/본문_크롤링/나무문화_본문_2022_7월.csv", encoding='UTF-8')
df

,url,본문
0,https://blog.naver.com/kuna611/222835896356,#25일급여 #삼성카드 #특판적금 #서울행복신협 #구즉신협 #카카오뱅크8%적금 #새...
1,https://blog.naver.com/zjavbxjqnd/222835899401,일요일 주말 휴가가 생겨서\n오빠랑 급 안동으로 여행 다녀왔어용\n(*ᴗ͈ˬᴗ͈)ꕤ...
2,https://blog.naver.com/roseyh78/222835897663,경주 켄싱턴리조트 퇴실 후 곧장 집으로 가자하면 어쩌나 했는데\n다행이 내가 가고 ...
3,https://blog.naver.com/y_girok_/222835898249,안녕-!\n재미없었고 별 다른 일이 없어서\n조용했던 저번주와는 달리\n시끌벅적 우...
4,https://blog.naver.com/anuyheel/222835899414,수영구에서 뭐 재미있는 거 배울만한 게 있을까?\n새로운 것을 도전해보면서 문화 생...
...,...,...
6214,https://blog.naver.com/jta0119/222824275603,"제주서쪽코스, 원데이 버스투어로 완성\n제주서쪽코스, 원데이 버스투어로 완성""\n안..."
6215,https://blog.naver.com/phronimosoo/222824275232,요즘 저희 아이가 음독하고 있는 책이에요.\n엄마표영어 2년 차때\n베렌스타인 베어...
6216,https://blog.naver.com/lee203kr/222824273297,산행지 : 중국 등봉시 숭산\n산행일 : 4박5일 일정중 4일차\n누구랑 : AM트...
6217,https://blog.naver.com/letteraunt/222824267410,아침운동으로 동네산책을 시작한 지 3개월이 되었다.\n몸무게를 재 보지는 않았지만 ...


In [6]:
len(df)

6219

In [9]:
#url_df.iloc[5349]

In [7]:
# 이건 필요할때 쓰면 됨. 블로그가 닫혔거나 삭제되었을때 빈 본문을 가진 데이터프레임 추가.
# content_dic = {'url' : ["https://blog.naver.com/himynet/222036795571"], '본문' : [None]}
# additional_df = pd.DataFrame(content_dic)
# df = df.append(additional_df, ignore_index = True)

C:\Users\user\AppData\Local\Temp\ipykernel_18064\2154302066.py:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(additional_df, ignore_index = True)


In [7]:
#### 삭제/비공개 링크 인덱스도 매번 바꿔줄 것!!! ###
# contents = []
# old_blogs = []
hide_link = []
waitsec = random.uniform(4,4.5)

for i in tqdm(blog_links[len(df):]):
    content_dic = {'url' : [], '본문' : []}
    content_dic['url'].append(i)
    driver.get(i)
    driver.implicitly_wait(100)
    time.sleep(waitsec)
    try:
        driver.switch_to.frame("mainFrame")
    except TimeoutException as e:
        print('시간 초과 오류 : ', e)
        content_dic['본문'].append('수집불가능')
    except UnexpectedAlertPresentException as e:
    ### 이 오류 해결함. 에러 코드 파일에서 직접 임포트하면 된다!
        print(e)
                                                                            # 여기도 바꿔야 함
        print('{}번째 게시글은 비공개로 전환되었거나 게시판이 바뀜.'.format(blog_links.index(i)))
        hide_link.append(i)
        time.sleep(1.5)
        driver.switch_to.frame('mainFrame')
    try:
        a = driver.find_element(By.CSS_SELECTOR,'div.se-main-container').text
        content_dic['본문'].append(a)
    # NoSuchElement 오류시 예외처리(구버전 블로그에 적용)
    except NoSuchElementException:
        a = driver.find_element(By.CSS_SELECTOR,'div#content-area').text
        content_dic['본문'].append(a)
        #old_blogs.append(i)
    additional_df = pd.DataFrame(content_dic)
    df = df.append(additional_df, ignore_index = True)
    df.to_csv("../dataset/2022/본문_크롤링/나무문화_본문_2022_7월.csv", index=False, encoding='UTF-8')
    #df.to_excel("./나무문화/2020/나무문화_본문_2020_7월.xlsx", index=False, encoding='UTF-8')
    # if blog_links.index(i) != df['본문'].tolist().index(a):
    #     break

C:\Users\user\AppData\Local\Temp\ipykernel_15140\3030760194.py:35: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(additional_df, ignore_index = True)
 17%|█▋        | 2202/12770 [7:31:55<112:55:50, 38.47s/it]C:\Users\user\AppData\Local\Temp\ipykernel_15140\3030760194.py:35: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(additional_df, ignore_index = True)
 17%|█▋        | 2203/12770 [7:32:03<86:19:25, 29.41s/it] C:\Users\user\AppData\Local\Temp\ipykernel_15140\3030760194.py:35: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(additional_df, ignore_index = True)
 17%|█▋        | 2204/12770 [7:32:10<65:56:33, 22.47s/it]C:\Users\user\AppData\Local\Temp\ipykernel_15140\3030760194.py:35: F

Alert Text: 비공개 글 입니다.
Message: unexpected alert open: {Alert text : 비공개 글 입니다.}
  (Session info: headless chrome=115.0.5790.171)

8468번째 게시글은 비공개로 전환되었거나 게시판이 바뀜.


C:\Users\user\AppData\Local\Temp\ipykernel_15140\3030760194.py:35: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(additional_df, ignore_index = True)
 18%|█▊        | 2250/12770 [7:39:32<23:47:35,  8.14s/it]C:\Users\user\AppData\Local\Temp\ipykernel_15140\3030760194.py:35: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(additional_df, ignore_index = True)
 18%|█▊        | 2251/12770 [7:39:40<23:21:39,  7.99s/it]C:\Users\user\AppData\Local\Temp\ipykernel_15140\3030760194.py:35: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(additional_df, ignore_index = True)
 18%|█▊        | 2252/12770 [7:39:46<21:59:53,  7.53s/it]C:\Users\user\AppData\Local\Temp\ipykernel_15140\3030760194.py:35: Fut

KeyboardInterrupt: 